In [1]:
import sys
import os

# Get the project directory
project_path = "/home/diego-ngz/Git/thesis-tabtrans"

sys.path.append(project_path) #This helps to be able to import the data from the parent directory to other files

from utils import tabtrans_file, data, training
import numpy as np
from skorch.callbacks import Checkpoint, TrainEndCheckpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint
import skorch
import torch.nn as nn
import torch

/home/diego-ngz/Git/thesis-tabtrans


In [2]:
'''
FINAL DATASETS
1484 lsvt
31 credit 
12 mfeat-factors 217 pass
20 mfeat-pixel 241 #have probelms this dataset check this one
9964 semeion 257 pass
233092 arrhythmia 280
3485 scene 300
9976 madelon 501
3481 isolet 618 (TO MUCH INSTANCES) 
'''

df_id = 31

X_train, X_test, y_train, y_test, train_indices, val_indices, n_instances, n_labels, n_numerical, n_categories = data.import_data(df_id)

INFO:openml.datasets.dataset:pickle write credit-g


In [3]:
n_layers = 3
n_heads = 2
embedding_size = 128
batch_size = 32
epochs = 25

#parameters of the NN
ff_pw_size = 30  #this value because of the paper
attn_dropout = 0.3 #paper
ff_dropout = 0.1 #paper value
aggregator = "cls"
aggregator_parameters = None
decoder_hidden_units = [128,64] #paper value
decoder_activation_fn = nn.ReLU()
need_weights = False
numerical_passthrough = False

#module
module = training.build_module(
    n_categories, # List of number of categories
    n_numerical, # Number of numerical features
    n_heads, # Number of heads per layer
    ff_pw_size, # Size of the MLP inside each transformer encoder layer
    n_layers, # Number of transformer encoder layers    
    n_labels, # Number of output neurons
    embedding_size,
    attn_dropout, 
    ff_dropout, 
    aggregator, # The aggregator for output vectors before decoder
    rnn_aggregator_parameters=aggregator_parameters,
    decoder_hidden_units=decoder_hidden_units,
    decoder_activation_fn=decoder_activation_fn,
    need_weights=need_weights,
    numerical_passthrough=numerical_passthrough
)


/home/diego-ngz/anaconda3/envs/tabtrans/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
epochs_to_save = [10,25]

for i in range(len(epochs_to_save)):
    print(i)
    print("Epoch: ", epochs_to_save[i])

In [4]:
epochs_to_save = [10,25,30]

train_end_cp = TrainEndCheckpoint(dirname = f"epoch_{epochs_to_save[0]}")

#create the model
model = skorch.NeuralNetClassifier(
        module = module,
        criterion=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.AdamW,
        device= "cuda", #cuda" if torch.cuda.is_available() else
        batch_size = batch_size,
        train_split = None,
        max_epochs = 10,
        optimizer__lr=1e-4,
        optimizer__weight_decay=1e-4,
        callbacks=[train_end_cp]
    )

model = model.fit(X={
    "x_numerical": X_train[:, :n_numerical].astype(np.float32),
    "x_categorical": X_train[:, n_numerical:].astype(np.int32)
    }, 
    y=y_train.astype(np.int64)
)

print("--------------------------------------------------------------------")

load_state = LoadInitState(train_end_cp)

train_end_cp = TrainEndCheckpoint(dirname = f"epoch_{epochs_to_save[1]}")

#create the model
model = skorch.NeuralNetClassifier(
        module = module,
        criterion=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.AdamW,
        device= "cuda", #cuda" if torch.cuda.is_available() else
        batch_size = batch_size,
        train_split = None,
        max_epochs = epochs_to_save[1]-epochs_to_save[0],
        optimizer__lr=1e-4,
        optimizer__weight_decay=1e-4,
        callbacks=[load_state, train_end_cp]
    )

model = model.fit(X={
    "x_numerical": X_train[:, :n_numerical].astype(np.float32),
    "x_categorical": X_train[:, n_numerical:].astype(np.int32)
    }, 
    y=y_train.astype(np.int64)
)


print("--------------------------------------------------------------------")
load_state = LoadInitState(train_end_cp)
train_end_cp = TrainEndCheckpoint(dirname = f"epoch_{epochs_to_save[2]}")

#create the model
model = skorch.NeuralNetClassifier(
        module = module,
        criterion=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.AdamW,
        device= "cuda", #cuda" if torch.cuda.is_available() else
        batch_size = batch_size,
        train_split = None,
        max_epochs = epochs_to_save[2]-epochs_to_save[1],
        optimizer__lr=1e-4,
        optimizer__weight_decay=1e-4,
        callbacks=[load_state, train_end_cp]
    )

model = model.fit(X={
    "x_numerical": X_train[:, :n_numerical].astype(np.float32),
    "x_categorical": X_train[:, n_numerical:].astype(np.int32)
    }, 
    y=y_train.astype(np.int64)
)

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6504  0.4109
      2        0.6098  0.2281
      3        0.6050  0.2271
      4        0.5901  0.2265
      5        0.5605  0.2269
      6        0.5346  0.2268
      7        0.5202  0.2273
      8        0.5101  0.2268
      9        0.5039  0.2265
     10        0.4991  0.2251
--------------------------------------------------------------------
  epoch    train_loss     dur
-------  ------------  ------
     11        0.4933  0.3164
     12        0.4902  0.3274
     13        0.4817  0.3261
     14        0.4831  0.3272
     15        0.4726  0.3253
     16        0.4655  0.3261
     17        0.4702  0.3261
     18        0.4603  0.3274
     19        0.4639  0.3273
     20        0.4550  0.3272
     21        0.4473  0.3267
     22        0.4577  0.3273
     23        0.4425  0.3264
     24        0.4463  0.3272
     25        0.4453  0.3258
-------------------------------------------------------------

In [21]:
# Define the model structure

model = skorch.NeuralNetClassifier(
        module = module,
        criterion=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.AdamW,
        device= "cuda", #cuda" if torch.cuda.is_available() else
        batch_size = batch_size,
        train_split = None,
        max_epochs = 10,
        optimizer__lr=1e-4,
        optimizer__weight_decay=1e-4,
        callbacks=[train_end_cp]
    )

# Initialize the model
model.initialize()

# Load the saved parameters
checkpoint_path = f"epoch_{epochs_to_save[0]}/train_end_params.pt"  # Adjust path as needed
model.load_params(f_params = checkpoint_path)


In [22]:
type(model)

skorch.classifier.NeuralNetClassifier

In [24]:
predictions = model.predict_proba(X={
        "x_numerical": X_test[:, :n_numerical].astype(np.float32),
        "x_categorical": X_test[:, n_numerical:].astype(np.int32)
        }
        )

In [ ]:
print(f"Instances: {X_train.shape[0]+X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")
print(f"F2i: {X_train.shape[1]/(X_train.shape[0]+X_test.shape[0])}")
print(f"Classes: {n_labels}")
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold with 5 splits
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 11)



In [ ]:
skf.split(X_train, y_train)

dict_split = {}
i = 1


for x,y in skf.split(X_train, y_train):

    dict_split[f"split_{i}"] = set(y)    
    i += 1

In [ ]:
'''
n_layers_lst = [2, 3, 4, 5] #2, 3, 4, 5
n_heads_lst = [4, 8, 16, 32] #4, 8, 16, 32
embed_dim = [128,256] #The embedding size is set one by one to avoid the out of memory error {128, 256}
batch_size = 32 # 32, 64, 128, 256, 512, 1024
epochs = 100
sample_size = [100,80,60,40,20]
'''
n_layers = 2
n_heads = 4
embedding_size = 128
batch_size = 32
epochs = 50


model, metrics = tabtrans_file.general_tabtrans(X_train, X_test, y_train, y_test, train_indices, val_indices, n_labels, n_numerical, n_categories, n_layers, n_heads, embedding_size, batch_size, epochs)


In [ ]:
metrics